In [2]:
from cmj.loa import models
from cmj.utils import Manutencao, run_sql
from django.conf import settings
from django.db.models import Q, F
from django.db.models.aggregates import Sum
from decimal import Decimal
import fitz
import re
m = Manutencao()
m.desativa_auto_now()
m.desativa_signals()

In [6]:
loa = models.Loa.objects.get(pk=2018)

In [7]:
(models.Despesa.objects.filter(loa=loa).delete(), 
models.Orgao.objects.filter(loa=loa).delete(),
models.UnidadeOrcamentaria.objects.filter(loa=loa).delete(),
models.Funcao.objects.filter(loa=loa).delete(),
models.SubFuncao.objects.filter(loa=loa).delete(),
models.Programa.objects.filter(loa=loa).delete(),
models.Acao.objects.filter(loa=loa).delete(),
models.Natureza.objects.filter(loa=loa).delete(),
models.EmendaLoaRegistroContabil.objects.filter(emendaloa__loa=loa).delete())

((0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {}),
 (0, {'loa.EmendaLoaRegistroContabil': 0}))

In [14]:
autografo = NormaJuridica.objects.get(pk=6926)
autografo

<NormaJuridica: Autógrafo nº 455 de 19 de Dezembro de 2017>

In [15]:
dfile = fitz.open(autografo.texto_integral.path)
#dfile = fitz.open('/home/leandro/Downloads/anexo_materia.pdf')

In [9]:
def load_pages_with(page_mask=''):
    rtext = ''
    for np in range(len(dfile)):
        page = dfile.load_page(np)
        text = page.get_text("text")

        if page_mask not in text:
            continue

        rtext += text
    while '  ' in rtext:
        rtext = rtext.replace('  ', ' ')
    while ' \n' in rtext:
        rtext = rtext.replace(' \n', '\n')
    while '\n\n' in rtext:
        rtext = rtext.replace('\n\n', '\n')
    return rtext

In [12]:
text_global = load_pages_with(page_mask='Quadro de Detalhamento da Despesa\nEXERCÍCIO DE 2018\n')

In [13]:
len(text_global), text_global[0:1000]

(0, '')

In [13]:
aplicacoes = []
for m in re.finditer(r'(.+)\n(\d{2}\.\d{3}\.\d{4}\.\d\.\d{3})', text_global):
    aplicacoes.append([m.start(), m.end(), list(m.groups())])
aplicacoes.append([len(text_global),len(text_global),None])
for i in range(len(aplicacoes)-1):
    aplic = aplicacoes[i]
    x1 = aplicacoes[i][1]
    x2 = aplicacoes[i+1][0]

    natdespesas = []

    for n in re.finditer(r'(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(\d{0,3}(?:\.\d{3})*,\d{2})', text_global[x1:x2]):
        natdespesas.append(list(n.groups()))

    aplic.append(natdespesas)
    
aplicacoes.pop()
aplicacoes = list(map(lambda x: (x[2][1], x[2][0], x[0], x[1], tuple(x[3])), aplicacoes))

In [14]:
aplicacoes

[('01.031.0139.1.001',
  'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
  329,
  382,
  (['4.4.90.51.00', '218.196,31'], ['4.4.90.52.00', '603.360,24'])),
 ('01.031.0139.2.001',
  'MANUT.ATIV.DA CAMARA MUNICIPAL',
  442,
  490,
  (['3.1.90.04.00', '9.486,80'],
   ['3.1.90.11.00', '10.785.818,60'],
   ['3.1.90.16.00', '18.973,59'],
   ['3.1.90.92.00', '948,68'],
   ['3.3.90.14.00', '137.558,55'],
   ['3.3.90.30.00', '512.287,00'],
   ['3.3.90.33.00', '6.640,76'],
   ['3.3.90.34.00', '151.788,74'],
   ['3.3.90.35.00', '47.433,98'],
   ['3.3.90.36.00', '85.381,17'],
   ['3.3.90.37.00', '9.486,80'],
   ['3.3.90.39.00', '569.207,77'],
   ['3.3.90.92.00', '948,68'])),
 ('01.271.2839.9.061',
  'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
  803,
  861,
  (['3.1.90.13.00', '958.267,07'],
   ['3.1.90.54.00', '1.897,36'],
   ['3.1.90.56.00', '1.897,36'],
   ['3.1.90.58.00', '474,34'],
   ['3.1.90.92.00', '474,34'])),
 ('01.272.2839.9.001',
  'ENC.PREV. C/O REGIME PRÓPRIO DE PREVIDÊNCIA',
  981,
  1042,
  

In [15]:
unidades_temp = []
for n in re.finditer(r'(?:CLASSIFICAÇÃO ORÇAMENTÁRIA\nVALORES\nDESPESA\nTOTAL\n)(.+)\n(.+)\n(.+)\n(.+)\n', text_global):
    unidades_temp.append([n.start(), n.end(), list(n.groups())])
    
unidades_temp.append([len(text_global),len(text_global),None])

In [16]:
unidades_temp[0]

[231, 329, ['CAMARA MUNICIPAL DE JATAI', 'CAMARA MUNICIPAL', '01', '01']]

In [17]:
unidades = []
for i in range(len(unidades_temp)-1):
    unidade = unidades_temp[i]
    x1 = unidades_temp[i][1]
    x2 = unidades_temp[i+1][0]
    
    unidade = [(unidade[2][2], unidade[2][0]), (unidade[2][3], unidade[2][1]), []]
    for aplic in aplicacoes:
        if aplic[2] >= x1 and aplic[3] <= x2:
            unidade[-1].append(aplic)    
    unidades.append(unidade)    

In [18]:
unidades_temp = unidades
unidades = []

orgao = ''
unidade_orc = ''
for u in unidades_temp:
    if u[0] != orgao or u[1] != unidade_orc:
        orgao = u[0]
        unidade_orc = u[1]
        unidades.append([orgao, unidade_orc, []])
    unidades[-1][-1].extend(u[2])

In [19]:
unidades

[[('01', 'CAMARA MUNICIPAL DE JATAI'),
  ('01', 'CAMARA MUNICIPAL'),
  [('01.031.0139.1.001',
    'OBRAS,INSTAL.EQUIP.CAMARA MUNICIPAL',
    329,
    382,
    (['4.4.90.51.00', '218.196,31'], ['4.4.90.52.00', '603.360,24'])),
   ('01.031.0139.2.001',
    'MANUT.ATIV.DA CAMARA MUNICIPAL',
    442,
    490,
    (['3.1.90.04.00', '9.486,80'],
     ['3.1.90.11.00', '10.785.818,60'],
     ['3.1.90.16.00', '18.973,59'],
     ['3.1.90.92.00', '948,68'],
     ['3.3.90.14.00', '137.558,55'],
     ['3.3.90.30.00', '512.287,00'],
     ['3.3.90.33.00', '6.640,76'],
     ['3.3.90.34.00', '151.788,74'],
     ['3.3.90.35.00', '47.433,98'],
     ['3.3.90.36.00', '85.381,17'],
     ['3.3.90.37.00', '9.486,80'],
     ['3.3.90.39.00', '569.207,77'],
     ['3.3.90.92.00', '948,68'])),
   ('01.271.2839.9.061',
    'ENC.PREV.C/O REGIME GERAL DE PREVIDÊNCIA',
    803,
    861,
    (['3.1.90.13.00', '958.267,07'],
     ['3.1.90.54.00', '1.897,36'],
     ['3.1.90.56.00', '1.897,36'],
     ['3.1.90.58.00', '474

# Funções e Subfunções

In [20]:
with open(f'funcao.txt', 'r') as fnd:
    funcoes = fnd.read()
    funcoes = re.split(r'\||\n', funcoes)
    funcoes = [(funcoes[i], funcoes[i+1]) for i in range(0, len(funcoes)-1, 2)]
    for c, e in funcoes:
        created, f = models.Funcao.objects.get_or_create(loa=loa, codigo=c, especificacao=e)
funcoes = sorted(funcoes, key=lambda x: (x[0], x[1]))
models.SubFuncao.objects.filter(loa=loa).count()

0

In [21]:
with open(f'subfuncao.txt', 'r') as fnd:
    subfs = fnd.read()
    subfs = re.split(r'\t|\n', subfs)
    subfs = [(subfs[i], subfs[i+1], subfs[i+2]) for i in range(0, len(subfs)-2, 3)]
    for fc, c, e in subfs:
        f = models.Funcao.objects.filter(loa=loa, codigo=fc).first()
        created, sf = models.SubFuncao.objects.get_or_create(loa=loa, funcao=f, codigo=c, especificacao=e)
subfs = sorted(subfs, key=lambda x: (x[0], x[1]))
models.SubFuncao.objects.filter(loa=loa).count()

114

In [22]:
count = [0,0,0]
for orgao, unidade, despesas in unidades:
    count[0] += 1
    for desp, espec, x1, x2, naturezas in despesas:
        count[1] += 1
        for nat in naturezas:
            count[2] += 1
print(count)

[28, 165, 740]


In [23]:
count = 0
for orgao, unidade, despesas in unidades:
    o_obj, created = models.Orgao.objects.get_or_create(
        loa=loa, codigo=orgao[0], especificacao=orgao[1]
    )

    uo_obj, created = models.UnidadeOrcamentaria.objects.get_or_create(
        loa=loa, orgao=o_obj, codigo=unidade[0], especificacao=unidade[1]
    )
    
    for desp, espec, x1, x2, naturezas in despesas:
        desp_re = re.match(r'(?P<funcao>\d{2})\.(?P<subfuncao>\d{3})'
                           r'\.(?P<programa>\d{4})\.(?P<acao>\d\.\d{3})', desp)

        funcao_obj = models.Funcao.objects.filter(loa=loa, codigo=desp_re['funcao']).first()
        subfunc_obj = models.SubFuncao.objects.filter(loa=loa, codigo=desp_re['subfuncao']).first()
        
        programa_obj, created = models.Programa.objects.get_or_create(loa=loa, codigo=desp_re['programa'])
        
        acao_obj, created = models.Acao.objects.get_or_create(loa=loa, codigo=desp_re['acao'], especificacao=espec)
        
        for nat in naturezas:
            count += 1
            natureza, created = models.Natureza.objects.get_or_create(loa=loa, codigo=nat[0])
            
            desp_obj, created = models.Despesa.objects.get_or_create(
                loa = loa,
                orgao = o_obj,
                unidade = uo_obj,
                funcao = funcao_obj,
                subfuncao = subfunc_obj,
                programa = programa_obj,
                acao = acao_obj,
                natureza = natureza)
            
            desp_obj.valor_materia = Decimal(nat[1].replace('.', '').replace(',', '.'))
            
            try: 
                desp_obj.save()
            except Exception as e:
                print(e)
                break
        if not desp_obj.pk:
            break
print(count, models.Despesa.objects.filter(loa=loa).count())

740 740


In [24]:
a = """text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.15 DE 20/06/78')
naturezas = []
for m in re.finditer(r'\n?(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n([^0-9]*)?\n?', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])
for esp, cod in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()
    """

# Natureza da Despesa

In [25]:
text_global = load_pages_with(page_mask='ADENDO III A PORTARIA SOF N.08 DE 04/02/85')
naturezas = []
for m in re.finditer(r'\n(\d{1,2}\.\d{1,2}\.\d{2}\.\d{2}\.\d{2})\n(.*)\n', text_global):
    naturezas.append(m.groups())
naturezas = sorted(naturezas, key=lambda x: x[0])

In [26]:
for cod, esp in naturezas:
    nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
    nat.especificacao=esp
    nat.save()

In [27]:
with open(f'2024_natureza_despesa.txt', 'r') as fnd:
    lines = fnd.read()
    lines = re.split(r'\t|\n', lines)
    naturezas = [(lines[i], lines[i+1]) for i in range(0, len(lines)-1, 2)] 
    for cod, esp in naturezas:
        nat, created = models.Natureza.objects.get_or_create(loa=loa, codigo=cod)
        if not nat.especificacao:
            nat.especificacao=esp
            nat.save() 

# Programas

In [28]:
text_global = load_pages_with(page_mask='ADENDO V A PORTARIA SOF N.15 DE 20/06/78')

In [29]:
programas = set()
for m in re.finditer(r'\n(\d{4})\n(.+)\n', text_global):
    programas.add(m.groups())

In [30]:
len(programas), models.Programa.objects.filter(loa=loa).values_list('codigo', flat=True).count()

(0, 22)

In [31]:
for cod, esp in list(programas):
    prog, created = models.Programa.objects.get_or_create(loa=loa, codigo=cod)
    prog.especificacao=esp
    prog.save()